In [1]:
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Software, Process, File,
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.definitions.stix21 import (
    ObservedData, IPv4Address, EmailAddress, DomainName, EmailMessage, URL, UserAccount, File,
    Identity, Incident, Note, Sighting, Indicator, Relationship, Location, Software, Process, Bundle
)
from stixorm.module.definitions.os_threat import (
    StateChangeObject, EventCoreExt, Event, ImpactCoreExt,
    Availability, Confidentiality, External, Integrity, Monetary, Physical,
    Traceability, Impact, IncidentScoreObject, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, ContactNumber, EmailContact,
    SocialMediaContact, IdentityContact, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from Block_Families.General._library.convert_n_and_e import convert_node, convert_relns, convert_sighting
from Block_Families.Context.Tab_Selections.get_unattached import get_unattached
import json
import os
from Utilities.util import conv
# 1. Variables
results_base = "../Orchestration/Results/examples/"
scratch_name = "scratch.json"
scratch = {}
final_nodes = []
final_ids = []
final_edges = []
s_nodes = []
s_edges = []
unattached = []
edges = []
relation_edges = []
relation_replacement_edges = []
relations = []
#
TR_Context_Memory_Dir = "./Context_Mem"
# UNATTACHED STUFF
# A. Software
software = Software(name="evil.exe")
s_nodes, s_edges = convert_node(conv(software))
# B. File
hashes ={"SHA-256": "fe90a7e910cb3a4739bed9180e807e93fa70c90f25a8915476f5e4bfbac681db"}
file = File(name="evil.exe", hashes=hashes)
nodes2, edges2 = convert_node(conv(file))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
print(f'file edges->', s_edges)
# C. Process
process = Process(pid=1221, created_time="2023-01-20T14:11:25.55Z",
                  command_line="./gedit-bin --destroy-alll", image_ref=file.id)
nodes2, edges2 = convert_node(conv(process))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
print(f'edges2 edges->', edges2)
print(f'process edges->', s_edges)


# A. Email Addresses and User Accounts
user_account3 = UserAccount(account_type="unix", account_login="sthor", display_name="silly thor")
nodes2, edges2 = convert_node(conv(user_account3))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
email_addr3 = EmailAddress(value="silly@mycompany.com", belongs_to_ref=user_account3.id)
nodes2, edges2 = convert_node(conv(email_addr3))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
user_account4 = UserAccount(account_type="unix", account_login="sguy", display_name="strange guy")
nodes2, edges2 = convert_node(conv(user_account4))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
email_addr4 = EmailAddress(value="strange@mycompany.com", belongs_to_ref=user_account4.id)
nodes2, edges2 = convert_node(conv(email_addr4))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
user_account5 = UserAccount(account_type="unix", account_login="dguy", display_name="dumbo guy")
nodes2, edges2 = convert_node(conv(user_account5))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
email_addr5 = EmailAddress(value="dumbo@mycompany.com", belongs_to_ref=user_account5.id)
nodes2, edges2 = convert_node(conv(email_addr5))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
# B. Duplicate-of SRO
sro4 = Relationship(relationship_type="duplicate-of", source_ref=email_addr3.id, target_ref=email_addr4)
nodes2, embedded, edges2, relation_replacement_edges = convert_relns(conv(sro4))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
print(f'\nembedded edges->', embedded)
print(f'\nedges2 edges->', edges2)
print(f'\nrelation_replacement_edges edges->', relation_replacement_edges)
sro5 = Relationship(relationship_type="duplicate-of", source_ref=email_addr3.id, target_ref=email_addr5)
nodes2, embedded, edges2, relation_replacement_edges = convert_relns(conv(sro5))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
email_message1 = EmailMessage(to_refs=email_addr3.id, from_ref=email_addr4.id, subject="we are coming for you",
                              is_multipart=False, date="2020-10-19T01:01:01.000Z")
nodes2, edges2 = convert_node(conv(email_message1))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
url1 = URL(value="https://www.northkorea.nk/we/are/mad/")
nodes2, edges2 = convert_node(conv(url1))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
rel1 = Relationship(relationship_type='derived-from', source_ref=email_message1.id, target_ref=url1.id)
nodes2, embedded, edges2, relation_replacement_edges = convert_relns(conv(rel1))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
# C. Setup observed-data
obs_refs2 = [email_addr3.id, user_account3.id, url1.id, rel1.id, email_message1.id,
             email_addr4.id, user_account4.id, email_addr5.id, user_account5.id,
             sro4.id, sro5.id]
observation3 = ObservedData(number_observed=1, object_refs=obs_refs2,
                            first_observed=email_message1.date,last_observed=email_message1.date)
nodes2, edges2 = convert_node(conv(observation3))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
#
# D. Setup Exchange Server Identity
exchange = Identity(name="Microsoft Exchange", description="Microsoft Exchange Server",
                    identity_class="system")
nodes2, edges2 = convert_node(conv(exchange))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
pat1 = "[email-addr:value = '" + email_addr3.value + "' AND email:subject = '" + email_message1.subject + "']"
ind1 = Indicator(name="Suspicious Email", pattern_type="stix", pattern=pat1, indicator_types=["unknown"])
nodes2, edges2 = convert_node(conv(ind1))
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2
#
# E. Setup Sighting with Context Evidence
query = "query from:"+email_addr3.value+", subject:"+email_message1.subject
value = "["+email_addr3.value+", "+email_addr4.value+", "+email_addr5.value+"]"
context = SightingContext(name="Exchange", description=query, value=value)
sight_ext = SightingEvidence(extension_type="property-extension")
sight_ext_id = "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43"
sight_context_ext = {
    sight_ext_id: sight_ext,
    "sighting-context": context
}
sighting3 = Sighting(observed_data_refs=observation3.id,
                     sighting_of_ref=ind1.id, extensions=sight_context_ext)
nodes2, edges2 = convert_sighting(conv(sighting3))
print("\n###############################################")
print("nodes2->", nodes2, "\n\n")
print("edges2->", edges2)
s_nodes = s_nodes + nodes2
s_edges = s_edges + edges2

scratch['nodes'] = s_nodes
scratch['edges'] = s_edges
with open(results_base + scratch_name, "w") as outfile:
    json.dump(scratch, outfile)


file edges-> []
edges2 edges-> [{'name': 'image-of', 'type': 'embedded', 'source': 'file--e7c7ddeb-d693-53e7-bcd8-f40951888120', 'target': 'process--04db2e3f-cec6-41f6-8945-269555e4af60', 'id': 'file--e7c7ddeb-d693-53e7-bcd8-f40951888120-process--04db2e3f-cec6-41f6-8945-269555e4af60'}]
process edges-> [{'name': 'image-of', 'type': 'embedded', 'source': 'file--e7c7ddeb-d693-53e7-bcd8-f40951888120', 'target': 'process--04db2e3f-cec6-41f6-8945-269555e4af60', 'id': 'file--e7c7ddeb-d693-53e7-bcd8-f40951888120-process--04db2e3f-cec6-41f6-8945-269555e4af60'}]

embedded edges-> []

edges2 edges-> [{'stix-id': 'relationship--64966c38-e47f-4429-b1c8-5778b8b04393', 'type': 'relationship', 'name': 'duplicate-of, from original', 'source': 'email-addr--a7155ffe-8ea7-563a-a54c-e94e23ec50c8', 'target': 'relationship--64966c38-e47f-4429-b1c8-5778b8b04393', 'id': 'email-addr--a7155ffe-8ea7-563a-a54c-e94e23ec50c8-relationship--64966c38-e47f-4429-b1c8-5778b8b04393'}, {'stix-id': 'relationship--64966c38-e4

In [2]:
# TREVIEW EXAMPLE
# Look into a tree and build edges
# if os.path.exists(results_base + sightings_file):
#     sighting_children = []
#     observed_children = []
#     # 1. Collect all of the nodes
#     with open(results_base + sightings_file, "r") as mem_input:
#         todal_edges = edges + reln_edges
#         sighting_tree_data = json.load(mem_input)
#         print(f'total edges->', todal_edges)
#         print(f'sighting_tree_data->', sighting_tree_data)
#         children = sighting_tree_data["children"]
#         sighting = children[0]
#         sighting_children = sighting["children"]
#         sighting["children"] = []
#         p_nodes.append(sighting)
#         for child in sighting_children:
#             if child["type"] == "observed-data":
#                 observed_children = child["children"]
#                 child["children"] = []
#                 p_nodes.append(child)
#             else:
#                 child["children"] = []
#                 p_nodes.append(child)
#         for child in observed_children:
#             child["children"] = []
#             p_nodes.append(child)
#         p_ids = [x["id"] for x in p_nodes]
#         pids = {x['id']:i for i, x in enumerate(p_nodes)}
#         p_edges = [x for x in todal_edges if x['source'] in p_ids and x['target'] in p_ids]
#         for p in p_nodes:
#             stix_index = p['id']
#             p['stix_id'] = stix_index
#             p['id'] =  pids[stix_index]
#         for p in p_edges:
#             source_val = p['source']
#             source_index = pids[source_val]
#             target_val = p['target']
#             target_index = pids[target_val]
#             p['stix_source'] = source_val
#             p['stix_target'] = target_val
#             p['source'] = source_index
#             p['target'] = target_index
